In [1]:
import numpy as np

def read_taillard_instance(file_path):
    """
    Read a Taillard instance from a file.

    Parameters:
    file_path (str): The path to the file containing the instance data.

    Returns:
    numpy.ndarray: A numpy array of shape (jobs, machines) containing the processing times
                   of each job on each machine.
    """

    with open(file_path, 'r') as f:
        # Read the number of jobs and machines
        line = f.readline().strip().split()
        jobs, machines = int(line[0]), int(line[1])

        # Read the processing times
        processing_time = np.zeros((jobs, machines))
        for j in range(jobs):
            line = f.readline().strip().split()
            for m in range(machines):
                processing_time[j, m] = int(line[m])

    return processing_time


In [2]:
class Node:
    def __init__(self, jobs, remaining_jobs, parent=None, lower_bound=None):
        self.jobs = jobs
        self.remaining_jobs = remaining_jobs
        self.parent = parent
        self.lower_bound = lower_bound

# Branch and Bound for FSP

In [3]:
def lower_bound(sequence, remaining_jobs, processing_times): 
    lower_bound = processing_times[sequence, 1:].max() + processing_times[list(remaining_jobs),:].sum(axis=0).max()
    return lower_bound
#sequence : assigned jobs
#Find the maximum processing time for the machines qui on des jobs


In [4]:
def evaluate_sequence(sequence, processing_times):
    machines = processing_times.shape[1]
    machine_times = np.zeros(machines)
    for job in sequence:
        machine_times[0] += processing_times[job][0]
        for i in range(1, machines):
            machine_times[i] = max(machine_times[i-1], machine_times[i]) + processing_times[job][i]
    return machine_times[-1]

In [15]:
import time
def branch_and_bound(processing_time,initial_sol,initial_cost): 
    time_limit = 30*60
    start_time = time.time()
    jobs,machines = processing_time.shape
    #init node list to the root_node
    root_node= Node([], set(range(jobs)))
    best_solution = initial_sol.copy()
    best_sol_cost = int(initial_cost)
    nodes = [root_node]
    #initial the best solution to range(jobs) and init best solution cost to it's cost
    i=1
    exec_time = 0
    while nodes and exec_time<time_limit: 
        
        node = nodes.pop()
        #explore neighbours
        for job in node.remaining_jobs:
            # ---- 
            
            child_jobs = node.jobs +[job]
            child_remaining_jobs = node.remaining_jobs - {job}
            # if child node is a leaf: calculate its cost 
            if not child_remaining_jobs : 
                child_lower_bound = evaluate_sequence(child_jobs,processing_time)
                if child_lower_bound < best_sol_cost : 
                    best_solution = child_jobs
                    best_sol_cost = child_lower_bound
                    continue
            else: 
                #if child node is not a leaf then calculate it's lower bound and compare it to the current best_sol_cost
                child_lower_bound = lower_bound(child_jobs, child_remaining_jobs, processing_time)

                if child_lower_bound< best_sol_cost : # Bound
                    
                    child_node = Node(child_jobs, child_remaining_jobs,parent = node , lower_bound=child_lower_bound)
                    nodes.append(child_node)

        i+=1
        time2 = time.time()
        exec_time = time2 - start_time

    return best_solution, best_sol_cost,i
    


## Read the Taillard instance

In [16]:
def processing_time(input_str):
    input_lines = input_str.strip().split('\n')
    n, m, seed, ub, lb = map(int, input_lines[0].strip().split())
    processing_times = []
    for line in input_lines[1:]:
        row = list(map(int, line.strip().split()))
        processing_times.append(row)
    return np.array(processing_times).transpose( )


In [42]:
input_str = '''\
8 3 379008056 1359 1290
 62 90 89 73 53 73 72 52 
 76 68 83 67 69 53 64 52
 89 69 62 81 82 68 87 58
'''
processing_times = processing_time(input_str)
print(processing_times)


[[62 76 89]
 [90 68 69]
 [89 83 62]
 [73 67 81]
 [53 69 82]
 [73 53 68]
 [72 64 87]
 [52 52 58]]


In [43]:
processing_times.shape

(8, 3)

In [44]:
#from the file
UB = 1278
LB = 1232

# Test

In [45]:
# Generate a random initial solution and calculate its cost
jobs = 8
initial_sol = list(np.random.permutation(jobs))
initial_cost = evaluate_sequence(initial_sol, processing_times)
print('init_cost is ', initial_cost)

init_cost is  734.0


In [46]:
best_solution, best_sol_cost,i = branch_and_bound(processing_times,initial_sol,initial_cost)


In [51]:
print("Soluton best cost for the random instance ",best_sol_cost)
print("Best solution :",best_solution)
print("iteration number",i)

Soluton best cost for the random instance  712.0
Best solution : [7, 4, 6, 5, 3, 1, 0, 2]
iteration number 69282


# Small test


In [103]:
input_str2 = '''\
20 5 873654221 1278 1232
54 83 15 71 77
79  3 11 99 56
16 89 49 15 89
'''
processing_times2 = processing_time(input_str2)
print(processing_times2)

[[54 79 16]
 [83  3 89]
 [15 11 49]
 [71 99 15]
 [77 56 89]]


In [110]:
# Generate a random initial solution and calculate its cost
jobs2 = 5
initial_sol2 = list(np.random.permutation(jobs2))
initial_cost2 = evaluate_sequence(initial_sol2, processing_times2)
print('init_cost is ', initial_cost2)

init_cost is  431.0


In [24]:
best_solution2, best_sol_cost2,i2 = branch_and_bound(processing_times2,initial_sol2,initial_cost2)



NameError: name 'processing_times2' is not defined

In [23]:
print(i2)
print(best_sol_cost2)

NameError: name 'i2' is not defined